In [ ]:
import sys#THIS IS FOR GOOGLE COLAB USERS ONLY
from google.colab import files
sys.path.insert(0,'/content/drive/MyDrive/IR-Project-Colab')

In [ ]:
!cp /content/drive/MyDrive/IR-Project-Colab/Utilities.py /content #THIS IS FOR GOOGLE COLAB USERS ONLY
!cp /content/drive/MyDrive/IR-Project-Colab/KNNImpute.py /content

In [ ]:
import Utilities  #THIS IS FOR GOOGLE COLAB USERS ONLY
import KNNImpute

In [ ]:
import nltk
import numpy as np
nltk.download('stopwords')
import pandas as pd
labels=["country", "sku_id", "title", "category_lvl1","category_lvl2","category_lvl3", "description", "price", "type"]

def getTrainingDataset():
    return pd.read_csv('data_train.csv', header=None, names=labels)  


def NullStatistics(df):
    missing_val = df.isnull().sum()
    print(missing_val)
    total_cells = np.product(df.shape)
    missing_percent = (missing_val.sum()/total_cells) *100
    print(f'\nThe missing data percent is: {missing_percent}')

trainingDataset=getTrainingDataset()

train_df,Y1,Y2,Y3=Utilities.Cleaning_Data_Utility(trainingDataset) #Utilities. hatadena yahan se 
# print(train_df)

NullStatistics(train_df)

unique_label_c1, unique_label_c2, unique_label_c3 = KNNImpute.preserve_label(train_df) #It has all unique values lying in ctg1 , 2 , 3 column

KNNImpute.encode(['category_lvl1', 'category_lvl2', 'category_lvl3'],train_df)        #Performed encoding for CTGLVL3 KNN

train_df_imputed = KNNImpute.impute(train_df)
train_df_imputed =  KNNImpute.clean_csv(train_df_imputed,train_df)
NullStatistics(train_df_imputed)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer   # Turning textual data into numeric for computation
X = train_df_imputed['Title_desc']
Y1 = train_df_imputed['category_lvl1']
Y2 = train_df_imputed['category_lvl2']
Y3 = train_df_imputed['category_lvl3']
print(Y1)
print(X)
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X_tfidf = vectorizer.transform(X)
print(X_tfidf)

0        2.0
1        3.0
2        7.0
3        3.0
4        3.0
        ... 
36278    1.0
36279    5.0
36280    1.0
36281    4.0
36282    7.0
Name: category_lvl1, Length: 36283, dtype: float64
0        adana galleri suri squar hijab light pink ul l...
1        cuba heartbreak eau de parfum spray ml oz form...
2        andoer cm cellphon smartphon mini dual head om...
3        anmyna complaint silki set shampoo ml conditio...
4        argit argiltubo green clay face bodi ml ul li ...
                               ...                        
36278    sade k led backlit wire usb mechan game keyboa...
36279    sona l electr oven seo ul li nbsp year warrant...
36280    op portabl wireless bluetooth speaker hand fre...
36281    woot woot tictacto pillow case white ul li cot...
36282    new smart wristband smart bracelet heart rate ...
Name: Title_desc, Length: 36283, dtype: object
  (0, 28913)	0.07225424844998868
  (0, 26966)	0.37659487115461027
  (0, 26075)	0.1998311858009805
  (0, 25125)

***CATEGORY 1 ***

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, Y2, test_size=0.3, random_state=42)
print(y_train)
print (X_train.shape)
print (X_test.shape)

SVM_Classfier=SVC(kernel='linear' , random_state=0)
SVM_Classfier.fit(X_train,y_train)
Y_Pred=SVM_Classfier.predict(X_test)
print(Y_Pred)

30176    56.0
22940    32.0
13208    32.0
15075    25.0
1106     45.0
         ... 
16850    56.0
6265      0.0
11284     1.0
860       5.0
15795    40.0
Name: category_lvl2, Length: 25398, dtype: float64
(25398, 31798)
(10885, 31798)
[27. 37. 27. ... 54.  0. 25.]


In [ ]:
from sklearn import metrics
# import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
# import seaborn as sns

print('Accuracy score with linear kernel is : ')
print(metrics.accuracy_score(y_test,Y_Pred)*100)
mat = confusion_matrix(y_test, Y_Pred)
print(mat)




Accuracy score with linear kernel is : 
84.12494258153423
[[1974    7    0 ...    0    3    7]
 [  29  282    0 ...    0    1    3]
 [   2    0   38 ...    0    0    6]
 ...
 [   7    0    0 ...  608    2    4]
 [  13    4    0 ...    6  124    2]
 [   3    2    0 ...    0    0 1167]]
